In [13]:
packages = ['pystac_client',
            'copernicusmarine',
            'xarray',
            'requests',
            'aiohttp',
            'copernicusmarine',
            'cartopy',
           'geopandas']

In [14]:
for package in packages:
    exec(f'import {package}')

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import geopandas
import cartopy.crs as ccrs
import json
import ast
from copernicusmarine.core_functions import custom_open_zarr
from shapely.geometry import Point, box, Polygon, shape
import geopandas as gpd
from shapely import wkt
from pystac_client import ItemSearch
from pystac import Item

## Reproduce error

In [27]:
url = 'https://catalog.dive.edito.eu'
client = pystac_client.Client.open(url)
print(client)
collections = list(client.get_collections())
variable1 = "sea_water_temperature"
for collection in collections:
    if variable1 in collection.id:
        print(collection.id)

<Client id=root>
climate_forecast-sea_water_temperature


In [28]:
products = []

for collection in collections:
    if variable1 in collection.id:
        for i, item in enumerate(collection.get_items()):
            products.append(item)

STACError: Invalid Item: If datetime is None, a start_datetime and end_datetime must be supplied.

## Find bad collection / Items

In [29]:
products = []
bad_items = []

for collection in collections:
    if variable1 in collection.id:
        try:
            # Create an ItemSearch for this collection
            search = ItemSearch(url=collection._items_href())  # raw items URL
            for item_dict in search.items_as_dicts():  # raw dicts
                props = item_dict.get("properties", {})
                dt = props.get("datetime")
                start_dt = props.get("start_datetime")
                end_dt = props.get("end_datetime")

                if dt is None and (start_dt is None or end_dt is None):
                    # Non-conforming item
                    bad_items.append({
                        "id": item_dict.get("id"),
                        "properties": props
                    })
                else:
                    # Safe to convert to Item
                    item = Item.from_dict(item_dict, root=collection)
                    products.append(item)
        except Exception as e:
            print(f"Error accessing collection {collection.id}: {e}")

print(f"Total valid items: {len(products)}")
print(f"Total invalid items: {len(bad_items)}")

# Inspect first few bad items
for bi in bad_items[:5]:
    print(bi)


Error accessing collection climate_forecast-sea_water_temperature: {"ErrorMessage":"Unauthorized","ErrorCode":401}
Total valid items: 0
Total invalid items: 0


## get info about the bad collection

In [30]:

collection_id = "climate_forecast-sea_water_temperature"

target_collection = None
for collection in collections:
    if collection.id == collection_id:
        target_collection = collection
        break

if target_collection is None:
    print(f"Collection '{collection_id}' not found.")
else:
    import json

    print("=== Full Collection Info ===")
    print("-" * 80)
    # extra_fields contains the full STAC metadata for CollectionClient
    # Pretty-print as JSON for readability
    print(json.dumps(target_collection.extra_fields, indent=4))

=== Full Collection Info ===
--------------------------------------------------------------------------------
{
    "type": "Collection",
    "version": "1.0.0",
    "aliases": [],
    "resto:info": {
        "model": "DTOModel",
        "lineage": [
            "DefaultModel",
            "DTOModel"
        ],
        "owner": "226726190540390401"
    },
    "properties": {
        "convention": {
            "name": "Climate Forecast convention",
            "url": "https://cfconventions.org",
            "standard_name": "sea_water_temperature",
            "aliases": [],
            "standard_name_table_version": "84",
            "amip": "to",
            "grib": "80",
            "canonical_units": "K"
        }
    },
    "@context": "https://schema.org/",
    "@type": "DataCatalog",
    "name": "Sea water temperature (Climate Forecast convention)",
    "identifier": "climate_forecast-sea_water_temperature",
    "isBasedOn": "https://api.dive.edito.eu/data/collections/climate_fo

In [ ]:
## code to s

In [38]:
from pystac_client import Client, ItemSearch
from pystac import Item

url = "https://catalog.dive.edito.eu"
client = Client.open(url)

variable1 = "sea_water_temperature"
collections = list(client.get_collections())

products = []

for collection in collections:
    if variable1 in collection.id:
        try:
            # Use ItemSearch to fetch raw item dicts
            search = ItemSearch(url=collection._items_href())
            for item_dict in search.items_as_dicts():
                # Convert to pystac Item safely without validation
                item = Item.from_dict(item_dict, root=collection, validate=False)
                products.append(item)

        except Exception as e:
            # Skip collections requiring authorization
            if "401" in str(e) or "Unauthorized" in str(e):
                print(f"Skipping {collection.id} (requires authorization)")
                continue
            else:
                # Raise any other unexpected exceptions
                raise

print(f"Total items fetched from public collections: {len(products)}")


Skipping climate_forecast-sea_water_temperature (requires authorization)
Total items fetched from public collections: 0
